<a href="https://colab.research.google.com/github/hayleypc/HawaiiClimate/blob/main/Hawaii_Soils_Process_and_Intersect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
install.packages('sf')
install.packages('raster')
install.packages('exactextractr')
install.packages("googledrive")
install.packages("googletoken")
install.packages("DBI")
install.packages("RSQLite")
install.packages("RPostgreSQL")
install.packages("terra")
install.packages('randomForest')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘proxy’, ‘e1071’, ‘wk’, ‘classInt’, ‘Rcpp’, ‘s2’, ‘units’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘sp’, ‘terra’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“package ‘googletoken’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the

In [2]:
library(ggplot2)
library(sf)
library(magrittr)
library(data.table)
library(raster)
library(exactextractr)
library(googledrive)
library(DBI) # database
library(terra)
library(randomForest)
library(parallel)

Linking to GEOS 3.11.1, GDAL 3.6.4, PROJ 9.1.1; sf_use_s2() is TRUE

Loading required package: sp

terra 1.7.71


Attaching package: ‘terra’


The following object is masked from ‘package:data.table’:

    shift


The following objects are masked from ‘package:magrittr’:

    extract, inset


randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:ggplot2’:

    margin




In [3]:
drive_auth(use_oob = TRUE, cache = FALSE)

Please point your browser to the following url: 

https://accounts.google.com/o/oauth2/v2/auth?client_id=603366585132-frjlouoa3s2ono25d2l9ukvhlsrlnr7k.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&redirect_uri=https%3A%2F%2Fwww.tidyverse.org%2Fgoogle-callback%2F&response_type=code&state=d4a14c433b242c826c0e29c5c73c7d08&access_type=offline&prompt=consent



Enter authorization code: eyJjb2RlIjoiNC8wQWRMSXJZZmlwNUV1al94S3VvMk9ORGNPMm1wZnpMbWswU3lfZzU3N2ltWld6NXItdGVQemlQNDhYd1NHVDl3V3libjl5ZyIsInN0YXRlIjoiZDRhMTRjNDMzYjI0MmM4MjZjMGUyOWM1YzczYzdkMDgifQ==


In [4]:
dir.create('/content/predictor_variables/')

In [5]:
# Hawaii GEDI data
# https://drive.google.com/file/d/1UY3lxML0Y7bat_9-xEZQQHDkYRcODDZy/view?usp=drive_link
drive_download(file = as_id('1UY3lxML0Y7bat_9-xEZQQHDkYRcODDZy'), path =  '/content/predictor_variables/Hawaii_GEDI_4A.gpkg', overwrite = FALSE)
gedi_sf <- st_read('/content/predictor_variables/Hawaii_GEDI_4A.gpkg')

File downloaded:

• Hawaii_GEDI.gpkg <id: 1UY3lxML0Y7bat_9-xEZQQHDkYRcODDZy>

Saved locally as:

• /content/predictor_variables/Hawaii_GEDI_4A.gpkg



Reading layer `Hawaii_GEDI_2' from data source 
  `/content/predictor_variables/Hawaii_GEDI_4A.gpkg' using driver `GPKG'
Integer64 values larger than 9.0072e+15 lost significance after conversion to double;
use argument int64_as_string = TRUE to import them lossless, as character
Simple feature collection with 4839466 features and 28 fields
Geometry type: POINT
Dimension:     XY
Bounding box:  xmin: -160.6006 ymin: 18.86556 xmax: -154.7559 ymax: 22.28996
Geodetic CRS:  WGS 84


In [28]:
# All islands 250m grid
# https://drive.google.com/file/d/1DcBOzWHwqJovd9hhIqLUjRlc2kpqEGKz/view?usp=drive_link
drive_download(file = as_id('1DcBOzWHwqJovd9hhIqLUjRlc2kpqEGKz'), path =  '/content/predictor_variables/HI_250_m_grid.gpkg', overwrite = FALSE)

Auto-refreshing stale OAuth token.

File downloaded:

• 250_summary_grid_dt_USE_THIS.gpkg <id: 1DcBOzWHwqJovd9hhIqLUjRlc2kpqEGKz>

Saved locally as:

• /content/predictor_variables/HI_250_m_grid.gpkg



In [8]:
folder_loc <- '/content/predictor_variables/TerraClimate/'
dir.create(folder_loc)

# https://drive.google.com/drive/folders/1bIbwKYmzP2o6VEAWyLIRmeMQbGFDt-vX?usp=drive_link
folder_id <- "1bIbwKYmzP2o6VEAWyLIRmeMQbGFDt-vX"
folder_files <- drive_ls(as_id(folder_id))

for (i in 1:nrow(folder_files)) {
  file_id <- folder_files$id[i]
  file_name <- folder_files$name[i]
  drive_download(as_id(file_id), path = paste0(folder_loc,file_name), overwrite = TRUE)
}


File downloaded:

• Hawaii_Mean_Climate_1958_to_2023.tif <id: 1pMs5OsTwQ1N3kXO9bb3W-qfhxwcJtzIj>

Saved locally as:

• /content/predictor_variables/TerraClimate/Hawaii_Mean_Climate_1958_to_2023.tif

File downloaded:

• Kauai_Mean_Climate_1958_to_2023.tif <id: 14bpiZID-VOQmsEVeQc9gFHe8JYK7w_ew>

Saved locally as:

• /content/predictor_variables/TerraClimate/Kauai_Mean_Climate_1958_to_2023.tif

File downloaded:

• Niihau_Mean_Climate_1958_to_2023.tif <id: 1T9RfMhwprSe0YV_ln5Xky9QTYycuh6M3>

Saved locally as:

• /content/predictor_variables/TerraClimate/Niihau_Mean_Climate_1958_to_2023.tif

File downloaded:

• Lanai_Mean_Climate_1958_to_2023.tif <id: 1eFkEW2ZRWtMK-IMEpVJ28R3xups_HhOT>

Saved locally as:

• /content/predictor_variables/TerraClimate/Lanai_Mean_Climate_1958_to_2023.tif

File downloaded:

• Maui_Mean_Climate_1958_to_2023.tif <id: 1Aglg9-srq187HYR38PFBVrodg3PpcqbE>

Saved locally as:

• /content/predictor_variables/TerraClimate/Maui_Mean_Climate_1958_to_2023.tif

File download

replace files in the Drive folder

In [9]:
folder_loc <- '/content/predictor_variables/SRTM_mTPI/'
dir.create(folder_loc)

# https://drive.google.com/drive/folders/1YKJrZXTo-L4UpvJ0iEGo8yIX6uhoQ9AW?usp=drive_link
folder_id <- "1YKJrZXTo-L4UpvJ0iEGo8yIX6uhoQ9AW"
folder_files <- drive_ls(as_id(folder_id))

for (i in 1:nrow(folder_files)) {
  file_id <- folder_files$id[i]
  file_name <- folder_files$name[i]
  drive_download(as_id(file_id), path = paste0(folder_loc,file_name), overwrite = TRUE)
}

File downloaded:

• Metadata <id: 1GB4d-4-xOOFHbf6R8z8yw5ZVYe4n-Oz-lSM43dpESb0>

Saved locally as:

• /content/predictor_variables/SRTM_mTPI/Metadata.docx

File downloaded:

• Lanai_SRTM_mTPI_Buffer.tif <id: 1sz3x9fZufpjbtrIpY3ZovhNknmDmkR5n>

Saved locally as:

• /content/predictor_variables/SRTM_mTPI/Lanai_SRTM_mTPI_Buffer.tif

File downloaded:

• Kauai_SRTM_mTPI_Buffer.tif <id: 1APv2WFDHdAiwOhBcAw8db-b1NSIlsK1K>

Saved locally as:

• /content/predictor_variables/SRTM_mTPI/Kauai_SRTM_mTPI_Buffer.tif

File downloaded:

• Niihau_SRTM_mTPI_Buffer.tif <id: 1JlGaWub7TJ31v7V1GSDQRIq5Mw4wcZx0>

Saved locally as:

• /content/predictor_variables/SRTM_mTPI/Niihau_SRTM_mTPI_Buffer.tif

File downloaded:

• Molakai_SRTM_mTPI_Buffer.tif <id: 1WRDZse9NGahMQ4h22bxvsv4XFp10JJZW>

Saved locally as:

• /content/predictor_variables/SRTM_mTPI/Molakai_SRTM_mTPI_Buffer.tif

File downloaded:

• Hawaii_SRTM_mTPI_Buffer.tif <id: 1Qej-icN4NhaYZ8fhhc4bkVXZuD385sl3>

Saved locally as:

• /content/predictor_varia

In [10]:
folder_loc <- '/content/predictor_variables/Elevation/'
dir.create(folder_loc)

# https://drive.google.com/drive/folders/1XmH9T9xVINZZfPFi39Wus1l7c0SKS4Am?usp=drive_link
folder_id <- "1XmH9T9xVINZZfPFi39Wus1l7c0SKS4Am"
folder_files <- drive_ls(as_id(folder_id))

for (i in 1:nrow(folder_files)) {
  file_id <- folder_files$id[i]
  file_name <- folder_files$name[i]
  drive_download(as_id(file_id), path = paste0(folder_loc,file_name), overwrite = TRUE)
}

File downloaded:

• Kauai_elevation.tif <id: 1SNmoJcKsyKSlG5DVye0c1pHvgWEQ5f1r>

Saved locally as:

• /content/predictor_variables/Elevation/Kauai_elevation.tif

File downloaded:

• Niihau_elevation.tif <id: 1L6lodPtr7u70scb-gu_QliaAIyEwYAtc>

Saved locally as:

• /content/predictor_variables/Elevation/Niihau_elevation.tif

File downloaded:

• Lanai_elevation.tif <id: 1XHgejTU-XLW51qDtVprOcGSeuINa4X77>

Saved locally as:

• /content/predictor_variables/Elevation/Lanai_elevation.tif

File downloaded:

• Hawaii_elevation.tif <id: 116xUvGPbJvJQrFZ7b59EzmcyAPHgKQmN>

Saved locally as:

• /content/predictor_variables/Elevation/Hawaii_elevation.tif

File downloaded:

• Maui_elevation.tif <id: 1Or-pj4QiOj8VgMOAPtVjMm_ocbrvGM6_>

Saved locally as:

• /content/predictor_variables/Elevation/Maui_elevation.tif

File downloaded:

• Oahu_elevation-0000000000-0000046592.tif
  <id: 1yRDrfvLTEGECTDQcwXRIxoHmUu3D9LL4>

Saved locally as:

• /content/predictor_variables/Elevation/Oahu_elevation-000000000

In [11]:
folder_loc <- '/content/predictor_variables/LandForms/'
dir.create(folder_loc)

# https://drive.google.com/drive/folders/1PkUR4GK50wC6Kgrvyy-d6067ypdX1dkx?usp=drive_link
folder_id <- "1PkUR4GK50wC6Kgrvyy-d6067ypdX1dkx"
folder_files <- drive_ls(as_id(folder_id))

for (i in 1:nrow(folder_files)) {
  file_id <- folder_files$id[i]
  file_name <- folder_files$name[i]
  drive_download(as_id(file_id), path = paste0(folder_loc,file_name), overwrite = TRUE)
}

File downloaded:

• Kauai_Landforms_Buffer.tif <id: 1oRmeE1kg5GswFcDdUaGU5jnui9Y2DpR3>

Saved locally as:

• /content/predictor_variables/LandForms/Kauai_Landforms_Buffer.tif

File downloaded:

• Oahu_Landforms_Buffer-0000000000-0000000000.tif
  <id: 1AzX2SjqLBZnxIfRwNAjzXQQuQGujN24p>

Saved locally as:

• /content/predictor_variables/LandForms/Oahu_Landforms_Buffer-0000000000-0000000000.tif

File downloaded:

• Lanai_Landforms_Buffer.tif <id: 1LU4LHzgkHHCmdOfWccniRJ8d0225AFu1>

Saved locally as:

• /content/predictor_variables/LandForms/Lanai_Landforms_Buffer.tif

File downloaded:

• Oahu_Landforms_Buffer-0000000000-0000065536.tif
  <id: 1PcZoQe0l8WLgSagiWWUpTmWmmHmGnIRx>

Saved locally as:

• /content/predictor_variables/LandForms/Oahu_Landforms_Buffer-0000000000-0000065536.tif

File downloaded:

• Molakai_Landforms_Buffer.tif <id: 1ZBdccF_ELdIApV0SkAv2ZfWsZ0a0ae1f>

Saved locally as:

• /content/predictor_variables/LandForms/Molakai_Landforms_Buffer.tif

File downloaded:

• Maui_Lan

In [12]:
folder_loc <- '/content/predictor_variables/LandSat_composite/'
dir.create(folder_loc)

# https://drive.google.com/drive/folders/1NzvurSghdEJdX0UsfdyuOcOyh6n3jUDo?usp=drive_link
folder_id <- "1NzvurSghdEJdX0UsfdyuOcOyh6n3jUDo"
folder_files <- drive_ls(as_id(folder_id))

for (i in 1:nrow(folder_files)) {
  file_id <- folder_files$id[i]
  file_name <- folder_files$name[i]
  drive_download(as_id(file_id), path = paste0(folder_loc,file_name), overwrite = TRUE)
}

File downloaded:

• Oahu_LS_composite_2012_2018-0000017920-0000026880.tif
  <id: 1R_TWiUhuLGchd-MWbL19NaY5wUSgNUwp>

Saved locally as:

• /content/predictor_variables/LandSat_composite/Oahu_LS_composite_2012_2018-0000017920-0000026880.tif

File downloaded:

• Oahu_LS_composite_2012_2018-0000000000-0000044800.tif
  <id: 1ig_nM95kDvLr8dgYlp0PQyx65YAYkAbI>

Saved locally as:

• /content/predictor_variables/LandSat_composite/Oahu_LS_composite_2012_2018-0000000000-0000044800.tif

File downloaded:

• Oahu_LS_composite_2012_2018-0000000000-0000026880.tif
  <id: 1x1OPChnSFwnKTsOqrBPopbJQKJTTmabl>

Saved locally as:

• /content/predictor_variables/LandSat_composite/Oahu_LS_composite_2012_2018-0000000000-0000026880.tif

File downloaded:

• Oahu_LS_composite_2012_2018-0000026880-0000017920.tif
  <id: 1M7xCSw8mfrToCz1W8hyk3bo2J4jYxLfn>

Saved locally as:

• /content/predictor_variables/LandSat_composite/Oahu_LS_composite_2012_2018-0000026880-0000017920.tif

File downloaded:

• Oahu_LS_composite_20

In [13]:
folder_loc <- '/content/predictor_variables/DynamicWorld_landcover/'
dir.create(folder_loc)

# https://drive.google.com/drive/folders/152pjTArpwlDVGjXFoSU9bmo1BDSKdCqq?usp=drive_link
folder_id <- "152pjTArpwlDVGjXFoSU9bmo1BDSKdCqq"
folder_files <- drive_ls(as_id(folder_id))

for (i in 1:nrow(folder_files)) {
  file_id <- folder_files$id[i]
  file_name <- folder_files$name[i]
  drive_download(as_id(file_id), path = paste0(folder_loc,file_name), overwrite = TRUE)
}

File downloaded:

• Oahu_Average_Land_Cover_Buffer_2000m_2015_to_2019-0000000000-0000074240.tif
  <id: 1Tnsd4PZQSPdEbdX3VtH-_doNBWuyWMtl>

Saved locally as:

• /content/predictor_variables/DynamicWorld_landcover/Oahu_Average_Land_Cover_Buffer_2000m_2015_to_2019-0000000000-0000074240.tif

File downloaded:

• Oahu_Average_Land_Cover_Buffer_2000m_2015_to_2019-0000000000-0000185600.tif
  <id: 19DtTLjlI_AVs4xBGyW5wjulRZljoOCC9>

Saved locally as:

• /content/predictor_variables/DynamicWorld_landcover/Oahu_Average_Land_Cover_Buffer_2000m_2015_to_2019-0000000000-0000185600.tif

File downloaded:

• Oahu_Average_Land_Cover_Buffer_2000m_2015_to_2019-0000007424-0000074240.tif
  <id: 1onONrmPMlNMtLjBkensOCDfagUTROlW2>

Saved locally as:

• /content/predictor_variables/DynamicWorld_landcover/Oahu_Average_Land_Cover_Buffer_2000m_2015_to_2019-0000007424-0000074240.tif

File downloaded:

• Oahu_Average_Land_Cover_Buffer_2000m_2015_to_2019-0000007424-0000089088.tif
  <id: 1K3KKn6GQhq1RKZaXwqaxCtl41QacT

In [14]:
  # https://drive.google.com/file/d/1ryTzWz0t9mgI5ynPvMzM21D135dLJvWv/view?usp=drive_link
  # Updated: https://drive.google.com/file/d/17c3DJcujQ_0MHirG2z2lSU88A0lvjo1C/view?usp=sharing
# drive_download(as_id('17c3DJcujQ_0MHirG2z2lSU88A0lvjo1C'), path = paste0('/content/fia.csv'), overwrite = TRUE)
# fia_dt <-  st_read('/content/fia.csv') %>% st_as_sf(coords = c("LON","LAT"), crs = 4326) %>% data.table

In [15]:
# fia_dt[,Stock_SOC_Mg_Ha :=  Stock_SOC_Mg_Ha %>% as.numeric,]

In [16]:
# (fia_dt %>% st_as_sf)[,'Stock_SOC_Mg_Ha' ] %>% plot

In [17]:
dir_list <- list.dirs('/content/predictor_variables')[-1]

names(dir_list) <- list.dirs('/content/predictor_variables',full.names = FALSE)[-1]

vrt is making a virtual raster tile
option to fix the issue: make a vrt individually for each island then stack them after
see if terraclimate is the issue (for Kauai), getting hidden by Oahu data

In [18]:
names(dir_list) %>% lapply(function(x){
  print(x)
 y <- dir_list[[x]]
 vrt(y %>% lapply(function(x){
  file_list <- list.files(y, pattern='*.tif',full.names = T)
  file_sizes <- lapply(file_list,file.size) %>% unlist %>% order
  file_list[file_sizes] %>% return
  }) %>% unlist,filename=paste0(y,'/',x,'.vrt'),overwrite=T)
 print(paste('Done',x))
})

[1] "DynamicWorld_landcover"
[1] "Done DynamicWorld_landcover"
[1] "Elevation"
[1] "Done Elevation"
[1] "LandForms"
[1] "Done LandForms"
[1] "LandSat_composite"
[1] "Done LandSat_composite"
[1] "SRTM_mTPI"
[1] "Done SRTM_mTPI"
[1] "TerraClimate"
[1] "Done TerraClimate"


[[1]]
[1] "Done DynamicWorld_landcover"

[[2]]
[1] "Done Elevation"

[[3]]
[1] "Done LandForms"

[[4]]
[1] "Done LandSat_composite"

[[5]]
[1] "Done SRTM_mTPI"

[[6]]
[1] "Done TerraClimate"

In [19]:
 raster_list <- list.files('/content/predictor_variables',pattern = '*.vrt',full.names = T,recursive = T) %>% lapply(stack) %>% lapply(rast)

raster_list
  # # Use gdalbuildvrt to create VRT
  # gdalbuildvrt(destfile = paste0(x,'/',x,'.vrt'),
  #              files =   (x %>% lapply(function(x){list.files(x, pattern='*.tif',full.names = T)})),
  #              separate = FALSE, # Change to TRUE if you want separate bands
  #              overwrite = TRUE)




[[1]]
class       : SpatRaster 
dimensions  : 367565, 2179623, 10  (nrow, ncol, nlyr)
resolution  : 9.136711e-05, 9.136711e-05  (x, y)
extent      : -178.4436, 20.70226, 18.86541, 52.44877  (xmin, xmax, ymin, ymax)
coord. ref. : +proj=longlat +datum=WGS84 +no_defs 
source      : DynamicWorld_landcover.vrt 
names       : Dynam~ver_1, Dynam~ver_2, Dynam~ver_3, Dynam~ver_4, Dynam~ver_5, Dynam~ver_6, ... 

[[2]]
class       : SpatRaster 
dimensions  : 35816, 87898, 1  (nrow, ncol, nlyr)
resolution  : 0.0002694946, 0.0002694946  (x, y)
extent      : -178.4436, -154.7556, 18.8652, 28.51742  (xmin, xmax, ymin, ymax)
coord. ref. : +proj=longlat +datum=WGS84 +no_defs 
source      : Elevation.vrt 
name        : Elevation 

[[3]]
class       : SpatRaster 
dimensions  : 80530, 270460, 1  (nrow, ncol, nlyr)
resolution  : 0.0002694946, 0.0002694946  (x, y)
extent      : -178.4437, -105.5562, 18.86543, 40.56783  (xmin, xmax, ymin, ymax)
coord. ref. : +proj=longlat +datum=WGS84 +no_defs 
source      :

In [20]:
gedi_dt <- st_read('/content/predictor_variables/Hawaii_GEDI_4A.gpkg') %>% data.table()

Reading layer `Hawaii_GEDI_2' from data source 
  `/content/predictor_variables/Hawaii_GEDI_4A.gpkg' using driver `GPKG'
Integer64 values larger than 9.0072e+15 lost significance after conversion to double;
use argument int64_as_string = TRUE to import them lossless, as character
Simple feature collection with 4839466 features and 28 fields
Geometry type: POINT
Dimension:     XY
Bounding box:  xmin: -160.6006 ymin: 18.86556 xmax: -154.7559 ymax: 22.28996
Geodetic CRS:  WGS 84


In [29]:
#https://drive.google.com/file/d/1DcBOzWHwqJovd9hhIqLUjRlc2kpqEGKz/view?usp=drive_link
grid_dt <- st_read('/content/predictor_variables/HI_250_m_grid.gpkg') %>% data.table()

Reading layer `250_summary_grid_dt' from data source 
  `/content/predictor_variables/HI_250_m_grid.gpkg' using driver `GPKG'
Simple feature collection with 359163 features and 35 fields
Geometry type: POINT
Dimension:     XY
Bounding box:  xmin: -159.7877 ymin: 18.91217 xmax: -154.807 ymax: 22.23237
Geodetic CRS:  WGS 84


In [22]:
# fia_dt[,c('Stock_SOC_Mg_Ha','geometry')]$Stock_SOC_Mg_Ha %>% qplot

In [23]:
#grid_int_dt <- fia_dt[,c('PLOT','Stock_SOC_Mg_Ha','geometry')] %>% st_as_sf() %>% st_transform(4135) %>% st_buffer(10000) %>% st_transform(4326) %>% st_intersects(grid_dt %>% st_as_sf %>% st_transform(4326))

In [25]:
#grid_int_unique <- grid_int_dt %>% unlist %>% unique

ERROR: Error in h(simpleError(msg, call)): error in evaluating the argument 'x' in selecting a method for function 'unique': object 'grid_int_dt' not found


In [27]:
# while(TRUE){}

In [26]:
# example_poly <- grid_dt[grid_int_unique,] %>% st_as_sf %>% st_buffer(250)

ERROR: Error: 'grid_int_unique' is not found in calling scope and it is not a column name either. When the first argument inside DT[...] is a single symbol (e.g. DT[var]), data.table looks for var in calling scope.


In [ ]:
# gedi_int_dt

In [ ]:
#grid_dt[grid_int_unique] %>% st_as_sf %>% st_write('grid_dt.gpkg',delete_dsn = T)

In [ ]:
  example_poly <- grid_dt %>% st_as_sf %>% st_transform(4135) %>% st_buffer(250) %>% st_transform(4326)

In [ ]:
  # example_poly <- (fia_extended_dt %>%data.table)[,c('PLOT','Stock_SOC_Mg_Ha','dummy_geom')]

  cover_dt <- data.table(exact_extract(raster_list[[1]],example_poly %>% st_as_sf,'mean',progress = TRUE) %>% data.table)
  names(cover_dt) <- c("water", "trees", "grass", "flooded_vegetation", "crops", "shrub_and_scrub", "built", "bare", "snow_and_ice",'max')

  elevation_dt <- data.table(exact_extract(raster_list[[2]],example_poly %>% st_as_sf,'mean',progress = TRUE) %>% data.table)
  names(elevation_dt) <- c("elevation")

  landform_dt <- data.table(exact_extract(raster_list[[3]],example_poly %>% st_as_sf,'mode',progress = TRUE) %>% data.table)
  names(landform_dt) <- c("landform")

  ls_dt <- data.table(exact_extract(raster_list[[4]],example_poly %>% st_as_sf,'mean',progress = TRUE) %>% data.table)
  names(ls_dt) <- c("ultra_blue", "blue", "green", "red", "infrared", "sw1", "sw2")

  tpi_dt <- data.table(exact_extract(raster_list[[5]],example_poly %>% st_as_sf,'mean',progress = TRUE) %>% data.table)
  names(tpi_dt) <- c("SRTM_mTPI")

  climate_dt <- data.table(exact_extract(raster_list[[6]],example_poly %>% st_as_sf,'mean',progress = TRUE) %>% data.table)
  names(climate_dt) <- c("aet", "def", "pdsi", "pet", "pr", "ro", "soil", "srad", "swe", "tmmn", "tmmx", "vap", "vpd", "vs")


In [ ]:
gedi_sf %>% data.table %>% names

In [ ]:
# st_crs(gedi_sf)
# grid_int_dt <- fia_dt[,c('PLOT','Stock_SOC_Mg_Ha','geometry')] %>% st_as_sf() %>% st_transform(4135) %>% st_buffer(10000) %>% st_transform(4326) %>% st_intersects(grid_dt %>% st_as_sf %>% st_transform(4326))

In [ ]:
# gedi_sf$l4_quality_flag %>% table
# gedi_dt <-gedi_sf %>% data.table
gedi_dt <- gedi_dt[l4_quality_flag == 1]

In [ ]:
gedi_int_dt <- gedi_dt$geom %>% st_intersects(example_poly,.)

In [ ]:
example_poly[1,] %>% st_intersects(gedi_sf)

In [ ]:
gedi_summary_dt <- gedi_int_dt %>% lapply(function(x){
  sub_gedi_dt <- gedi_dt[x,]
  sub_gedi_dt[,list(agbd_m = mean(agbd,na.rm=T),agbd_sd = sd(agbd,na.rm=T),agbd_n = length(agbd))]
}) %>% rbindlist()

In [ ]:
# gedi_summary_dt %>% nrow

In [ ]:
  rf_dt <- data.table(example_poly,cover_dt,elevation_dt,landform_dt,tpi_dt,climate_dt,gedi_summary_dt)


In [ ]:
  rf_dt$geom <- grid_dt[grid_int_unique]$geom


In [ ]:
rf_dt %>% st_as_sf %>% st_write('250_summary_grid_dt.gpkg',delete_dsn = T)

In [ ]:
rf_dt %>% names

In [ ]:
rf_dt %>%
  ggplot(aes(x=srad,y=agbd_m)) +
  # xlim(0,500)+

  geom_bin2d() +
    geom_smooth(method='lm')

In [ ]:
# https://drive.google.com/drive/folders/1uVZCzCb5gRE8wfZckspRuKz1TQd5LpeM?usp=drive_link
drive_upload("/content/250_summary_grid_dt.gpkg", path = as_id("1uVZCzCb5gRE8wfZckspRuKz1TQd5LpeM"))